In [ ]:
import glob
import cv2
import numpy as np
import os 
from skimage.io import imread
i = 1
scale=300
class_list=["0","1","2","3","4"]
for folder in range(5):
    for root, dirs, files in os.walk(r"D:/diabetic-retinopathy-detection/train/train/"+str(class_list[folder])):
        for file in files:
            if file.endswith('.jpeg'):   
                imgpath = root+"/"+file
                img = cv2.imread(imgpath)
                if img.size != 0:
                    grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                    _,th2 = cv2.threshold(grey,8,255,cv2.THRESH_BINARY)
                    contours, hierarchy = cv2.findContours(th2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                    areas = [cv2.contourArea(contour) for contour in contours]
                    max_index = np.argmax(areas)
                    cnt = contours[max_index]
                    x,y,w,h = cv2.boundingRect(cnt)
                    # Ensure bounding rect should be at least 16:9 or taller
                    if w / h > 16 / 9:
                   # increase top and bottom margin
                        newHeight = w / 16 * 9
                        y = y - (newHeight - h ) / 2
                        h = newHeight
                # Crop with the largest rectangle
                
                    crop = img[int(y):int(y+h),int(x):int(x+w)] 
                    try:
                        img = cv2.resize(crop,(512,512))
                        a=cv2.addWeighted(img,4,cv2.GaussianBlur(img,(0,0),scale/30),-4,128)
                        b=np.zeros(a.shape)
                        cv2.circle(b,(a.shape[1]//2,a.shape[0]//2),
                        int(scale*0.9),(1,1,1), -1,8,0)
                        a=a*b+128*(1-b)
                        print(imgpath)
                        cv2.imwrite(imgpath,a)
                        cv2.waitKey(0)  
                        #print(str(i))
                        i = i+1 		    
                    except:
                        pass